In [1]:
import json
import pandas as pd

In [2]:
likes_path = "data/easytour-likes-2022-03MAR-21.json"
schede_path = "data/easytour-schede-2022-03MAR-21.json"

In [3]:
f_schede = open(schede_path)
data_schede = json.load(f_schede)
print("Fields: {}".format(data_schede[0].keys())) 
print("Number of samples: {}".format(len(data_schede)))

Fields: dict_keys(['id', 'userId', 'userName', 'documentType', 'title', 'description', 'locations', 'feature', 'duration', 'imageId', 'previewId', 'distance', 'datePublishing', 'widgets', 'counterUseful', 'tags', 'numberOfPositions', 'creationDate', 'videoId', 'version', 'categories', 'media', 'shareType', 'status', 'lastUpdate', 'versionHistory', 'viewCounter', 'device'])
----------------------------------------------------------------------------------------------------
Number of samples: 4356


In [20]:
#print(data_schede[0])

In [5]:
f_likes = open(likes_path)
data_likes = json.load(f_likes)
print("Number of samples: {}".format(len(data_likes)))
print("Fields: {}".format(data_likes[0].keys()))
print("First sample: {}".format(data_likes[0]))

Number of samples: 1503
----------------------------------------------------------------------------------------------------
Fields: dict_keys(['id', 'userId', 'userName', 'lastUpdate'])
----------------------------------------------------------------------------------------------------
{'id': '622c3b8ac992ca28f33c7f21', 'userId': 606, 'userName': 'Dale On Travel', 'lastUpdate': '2022-03-19T07:56:20.868Z'}


In [6]:
df_schede = pd.read_json(schede_path)
df_likes = pd.read_json(likes_path)

In [7]:
df_schede_text = df_schede[["id","userId", "description"]]
df_likes_ids = df_likes[["id", "userId"]]

In [8]:
df_schede_text

,id,userId,description
0,62355f98272ae5672fbc3ac7,632,"Phuket, il paradiso per gli amanti del sole, d..."
1,620abb33ff8ae67f37cafe3c,442,Forse la spiaggia più bella di tutta l’isola d...
2,620aa536ff8ae67f37cafb06,442,So che dietro queste cose c’è spesso del marci...
3,62349ff2272ae5672fbc3135,632,Phuket è conosciuta per essere la meta predile...
4,6234aa65272ae5672fbc34c2,632,"Phuket non è solo spiagge, ci sono anche altri..."
...,...,...,...
4351,61f57f62306802033034ceed,536,Ripartiamo dopo la nostra esperienza in Amazzo...
4352,61f668fa306802033034d239,536,E così ci lasciamo alle spalle anche la splend...
4353,620bbf26b1675d6cc11232f8,431,"La chiamano ""ciudad gris"" Lima. Per circa quat..."
4354,61f3c6add91e9928f6e0922a,431,È quasi impossibile trovare un angolo del Perù...


In [9]:
#One description is empty
df_schede_text.dropna(inplace=True)
len(df_schede_text)

/tmp/ipykernel_3857/3866547532.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_schede_text.dropna(inplace=True)


4355

In [10]:
import string
from typing import *

import nltk
import unidecode
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer

#Downloads for stopwords and punctuation
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('italian')
nltk.download('wordnet')
string.punctuation

#Removing accented chars
def remove_accented_chars(text: string):
    text = unidecode.unidecode(text)
    return text

#Removing stopwords
def remove_stopwords(text: string):
  output = [i for i in text.split() if i not in stopwords]
  return output

#Removing punctuation
def remove_punctuation(text: string):
  list_without_punctuation = []
  for i in text:
    if i not in string.punctuation:
      list_without_punctuation.append(i)
    else:
      list_without_punctuation.append(' ')
  string_without_punctuation = ''.join(list_without_punctuation)
  return string_without_punctuation

#Removing numbers
def remove_numbers(text: string):
  list_without_numbers = []
  for i in text:
    if i not in '0123456789':
      list_without_numbers.append(i)
    else:
      list_without_numbers.append(' ')
  string_without_numbers = ''.join(list_without_numbers)
  return string_without_numbers

#Lemmatization
def lemmatizer(text: string):
    wordnet_lemmatizer = WordNetLemmatizer()
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text

#Stemming
def stemming(text: string):
    italian_stemmer = SnowballStemmer('italian')
    stem_text = [italian_stemmer.stem(word) for word in text if not word.isdigit()]
    return stem_text

def preprocess(text: string, accent=0, punct=1, numb=1, stop=1, lemma=0, stem=1):
  """Preprocess the data using this function
  Args:
      text (string): text to be preprocessed
      accent (int, optional): to delete accented chars. Defaults to 0.
      punct (int, optional): to delete punctuation. Defaults to 1.
      numb (int, optional): to delete numbers. Defaults to 1.
      stop (int, optional): to delete stopwords. Defaults to 1.
      lemma (int, optional): to apply lemmatization. Defaults to 0.
      stem (int, optional): to apply stemming. Defaults to 1.
  Returns:
      [type]: preprocessed text
  """
  text = text.lower()
  if accent==1:
      text = remove_accented_chars(text)
  if punct==1:
      text = remove_punctuation(text)
  if numb==1:
      text = remove_numbers(text)
  if stop==1:
      text = remove_stopwords(text)
  if lemma==1:
      text = lemmatizer(text)
  if stem==1:
      text = stemming(text)
  return text

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/gianfree/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/gianfree/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
#df_schede_description = df_schede_text.description.apply(lambda x: preprocess(x))

In [12]:
df_schede_text

,id,userId,description
0,62355f98272ae5672fbc3ac7,632,"Phuket, il paradiso per gli amanti del sole, d..."
1,620abb33ff8ae67f37cafe3c,442,Forse la spiaggia più bella di tutta l’isola d...
2,620aa536ff8ae67f37cafb06,442,So che dietro queste cose c’è spesso del marci...
3,62349ff2272ae5672fbc3135,632,Phuket è conosciuta per essere la meta predile...
4,6234aa65272ae5672fbc34c2,632,"Phuket non è solo spiagge, ci sono anche altri..."
...,...,...,...
4351,61f57f62306802033034ceed,536,Ripartiamo dopo la nostra esperienza in Amazzo...
4352,61f668fa306802033034d239,536,E così ci lasciamo alle spalle anche la splend...
4353,620bbf26b1675d6cc11232f8,431,"La chiamano ""ciudad gris"" Lima. Per circa quat..."
4354,61f3c6add91e9928f6e0922a,431,È quasi impossibile trovare un angolo del Perù...


<h3>Remember to remove verbs and adjectives!</h3>

In [13]:
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# df_pos = df_schede_text.description.apply(lambda x: nltk.pos_tag(nltk.word_tokenize(x)))
# df_pos

http://rdrpostagger.sourceforge.net/

In [14]:
# import RDRPOSTagger
# r = RDRPOSTagger()
# r.constructSCRDRtreeFromRDRfile("../Models/POS/Italian.RDR") # Load POS tagging model for Italian language
# DICT = readDictionary("../Models/POS/Italian.DICT") # Load a Italian lexicon

# df_pos = df_schede_text.description.apply(lambda x: r.tagRawSentence(DICT, x))


<h3>Recommender system</h3>

In [17]:
#list of user
users = pd.concat([df_likes_ids.userId, df_schede_text.userId]).drop_duplicates()
#list of posts
posts = df_schede_text.id
#list of users and posts they liked
users_likes = df_likes_ids.groupby("userId").id.apply(list)
users_likes

userId
114    [61447a3224c29e59fe4c06ab, 61824f5f21889d3548f...
119    [6172da4472b6f336088b932b, 6218ec9f29ef4a75805...
129    [615acf21eb3bf47494b8e64b, 61aa039788b2e606829...
149    [61faf5d953a6a72116188d5b, 61f41fe330680203303...
150    [61b72f34e364b82f2ac8c88c, 61f3c801d91e9928f6e...
                             ...                        
630    [62215a990a0ebd53f7469748, 62174fea99163528c06...
632                           [6229df13aeb26768dc408e81]
637    [6212993a4f9d177c90f3830f, 6224bdb835684f072b2...
639                           [620a39a4e7ce7d5992612316]
640    [622aeada99fd4c40442f3082, 6217ab40ae6735171d5...
Name: id, Length: 116, dtype: object

In [18]:
#initializing tfidf vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(lowercase=False)

tfidf_schede = tfidf_vectorizer.fit_transform((df_schede_text['description'])) 
tfidf_schede

<4355x59290 sparse matrix of type '<class 'numpy.float64'>'
	with 718826 stored elements in Compressed Sparse Row format>

In [19]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(tfidf_schede, tfidf_schede)

In [55]:
def get_recommendations(user_id: int):
    # Get the index of the user that we want to recommend posts to
    user_id_idx = df_schede[df_schede == user_id].index[0]
    # Get the pairwsie similarity scores of all posts with that user
    sim_scores = list(enumerate(cosine_sim[user_id_idx]))
    # Sort the posts based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Get the scores of the 10 most similar posts
    sim_scores = sim_scores[1:11]
    # Get the blog post ids
    post_ids = [i[0] for i in sim_scores]
    # Return the blog posts
    return df_schede.iloc[post_ids]

def print_recommendations(recommended_posts):
    for i in range(len(recommended_posts)):
        print("Post #" + str(i+1) + ":")
        print("Post ID: " + str(recommended_posts.iloc[i].id))
        print("Title: " + str(recommended_posts.iloc[i].title))
        print("Description: " + str(recommended_posts.iloc[i].description))
        print("\n")

In [57]:
id = 632
#print("Posts that user {} liked:".format(id))
liked_posts = df_likes_ids[df_likes_ids["userId"] == id]
#print(liked_posts)
print("Posts:")
for _,p in liked_posts.iterrows():
    post = df_schede[df_schede["id"]== p["id"]]
    if not post.description.empty:
        print(post.title)
        print("-"*100)

Posts:
92    Bemo - immergiti nella vita locale indonesiana
Name: title, dtype: object
----------------------------------------------------------------------------------------------------


In [58]:
recommended_posts = get_recommendations(id)
print_recommendations(recommended_posts)

Post #1:
Post ID: 620abb33ff8ae67f37cafe3c
Title: Freedom beach
Description: Forse la spiaggia più bella di tutta l’isola di phuket! Freedom beach si trova vicino la spiaggia di patong ed è possibile raggiungerla solo via mare perché è abbastanza complicato arrivarci via terra, per la foresta che circonda la spiaggia! Proprio per questo nella spiaggia vicina troverete che fanno servizio taxi, tanti locali che vi porteranno sulla spiaggia in barca e aspetteranno li in barca mentre voi vi godete la giornata di sole e farete il bagno nell’acqua cristallina dell’oceano!


Post #2:
Post ID: 620d2de325398565dadb6ad2
Title: Il paradiso dei sub 
Description: Sembra il paradiso ma si tratta della spiaggia di Zambrone, in Calabria. Una delle spiagge più bella del Tirreno, un mare stupendo e una spiaggia poco affollata e ancora del tutto selvaggia. Raggiungere la spiaggia non è facilissimo, bisogna lasciare la macchina e percorrere un sentiero a piedi per giungere alla spiaggia, ma ne vale davver